# Instructions
The following code was designed in order to track the location of a single animal across the course of a single video session.  After initally loading in the video, the user is able to crop the video frame using a rectangle selection tool.  A background reference frame is then specified, either by taking a median of several frames in the video, or by the user providing a short video of the same environment without an animal in it.  By comparing each frame in the video to the reference frame, the location of the animal can be tracked.  It is imperative that the reference frame of the video is not shifted from the actual video.  For this reason, if a separate video is supplied, it is best that it be acquired on the same day as the behavioral recording.  The animal's center of mass, in x,y coordinates, is then recorded, as well as the distance in pixels that the animal moves from one frame to the next. Lastly, the user can specify regions of interest in the frame (e.g. left, right) using a polygon drawing tool and record for each frame whether the animal is in the region of interest.  Options for summarizing the data are also provided. 

---
# 1. Load Necessary Packages
The following code loads neccessary packages and need not be changed by the user.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import holoviews as hv
import numpy as np
import pandas as pd
import LocationTracking_Functions as lt

import cv2
import math
import re
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib.image as mpimg
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

---
# 2. User Sets Directory and File Information, and Specifies ROI Names, if any.
`dpath` : The directory path of the folder containing the video to be processed. Note that if you are using a Windows path with backslashes, place an ‘r’ in front of the directory path to avoid an error (e.g., r’\Users\DeniseCaiLab\Videos’).

`file` : The filename of the video, including the file extension.

`start` : The frame of the video on which to begin processing.  0 is the first frame. 

`end` : The frame of the video on which to end processing.  If the user would like to process from the start frame to the end of the video, this can be set to None.

`region_names` : If the user would like to measure the time spent in ROIs, a list containing the names of the ROIs should be provided.  A Python list is defined by a set of square brackets, and each ROI name should be placed in quotations, separated by a comma.  If no ROIs are to be defined, this can be set to None (i.e., `‘region_names’ : None`).

`dsmpl` : The amount to down-sample each frame. A value of 1 indicates no down-sampling, while a value of 0.25 indicates that each frame will be down-sampled to ¼ its original size.  Note that if down-sampling is performed, all pixel coordinate output will be in the dimensions of the down-sampled video.

`stretch` : Allows the user to alter the aspect ratio of the presented output.  This is useful when videos have irregular dimensions and are difficult to see (e.g., an aspect ratio of 1:100).  The width/height will be scaled by the factor provided. Note that this only affects the appearance of visualizations and does not modify the video or the interpretation of the output.

***Processing going slow?  Consider downsampling!***  Often times tracking does not not require 1080p or whatever high def resolution videos are sometimes acquired using. Try setting `dsmpl` to something lower than 1 to implement downsampling.

In [ ]:
input_dir = "/Users/hcjiang/Dropbox (UMass Medical School)/olfactory behavior/habituation"
output_dir = "/Users/hcjiang/Desktop/ezTrack/ezTrack output/" 
video_name = '20220207_mouae6_cd20 ko_female15w_1-hab.mp4'
prefix = re.sub('.mp4', '',video_name)


In [ ]:
folders = ["location_raw","location_trace","heatmap","location_trace_central_distance","distance_across_session","summary"]
output_path = []
for i in folders:
    newpath = output_dir + "/" +i
    output_path.append(newpath)
    if not os.path.exists(newpath):
        os.makedirs(newpath)

In [ ]:
output_path

In [ ]:
video_dict = {
    'dpath'        : input_dir,  
    'file'         : video_name,
    'start'        : 0, 
    'end'          : 45553, 
    'region_names' : ['avoidance zone','odorized zone','odor'],
    'dsmpl'        : 0.25,
    'stretch'      : dict(width=1, height=1)
}

---
# 3. Load Video and Crop Frame if Desired
To crop video frame, after running code below, select the box selection tool below the image (square with a plus sign).  To start drawing region to be included in the analyis, double click image.  Double click again to finalize region.  If you decide to change region, it is best to rerun this cell and subsequent steps.

If the size of the image is too small/large, alter the first line of code.  100 is the standard size.  200 will produce an image 2x the size, and so on.

In [ ]:
%%output size = 200

img_crp, video_dict = lt.LoadAndCrop(video_dict, cropmethod='Box')
img_crp

# 4. (Optional) Mask Internal Regions

The following code is used to exclude internal portion/s of the field of view from the analysis. After running cell below, draw regions to be excluded.  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell. 

In [ ]:
%%output size = 200

img_mask, video_dict['mask'] = lt.Mask_select(video_dict)
img_mask

---
# 5. Define Reference Frame for Location Tracking.
For location tracking to work, a view of box without animal must be provided.  Below there are two ways to do this.  **Option 1** provides a method to remove the animal from the video.  This option works well provided the animal doesn't stay in the same location for >50% of the session. Alternatively, with **Option 2**, the user can simply define a video file in the same folder that doesn't have in animal in it.  Option 1 is generally recommended, being simpler to obtain. 

The number of random frames used can be changed with the variable `num_frames`.  Alternatively, if the user would like to manually define the frames to be used, `frames` can be set to a list frame numbers: e.g. `frames = np.arange(100,500,5)`, which would select every 5th frame in the range 100-500).  Setting `frames` will override the num_frames variable selection. 

### Option 1 - Create reference frame by removing animal from video
The following code takes a random sample of frames across the session and creates an average of them by taking the median for each pixel.  

In [ ]:
%%output size = 200

video_dict['reference'], img_ref = lt.Reference(video_dict, num_frames=50, frames=None) 
img_ref

### Option 2 - User specifies video of empty box
The following code allows the user to specify a different file.  Set video_dict['altfile'] to the alternative filename.  

Defining `frames` is necessary if the alternative video is a different length than the reference video.

In [ ]:
%%output size = 100

video_dict['altfile'] = 'EmptyBox.avi' 

video_dict['reference'], img_ref = lt.Reference(video_dict, num_frames=50, altfile=True, frames=[0]) 
img_ref

# 6. (Optional) Use Interactive Plot to Define Regions of Interest.  

After running cell below, draw regions of interest on presented image in the order you provided them (in Cell 2).  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell.

***Note*** that there are no problems if regions overlap.

In [ ]:
%%output size = 200


img_roi, video_dict['roi_stream'] = lt.ROI_plot(video_dict)
img_roi

---
# 7. (Optional) Define Scale for Distance Calculations

### 7a. Select two points of known distance

After running cell below, click on any two points and the distance between them, in pixel units, will be presented. Will be used to convert pixel distance to other scale. Note that once drawn, points can be dragged or you can click again.

In [ ]:
%%output size = 200

img_scl, video_dict['scale'] = lt.DistanceTool(video_dict)
img_scl

### 7b. Define real-world distance between points
Below, define the distance between the points selected, and the scale. Note that scale can be any desired text.

In [ ]:
distance = 10
scale = 'cm'

video_dict['scale'] = lt.setScale(distance, scale, video_dict['scale'])

---
# 8. Track Location

### 8a. Set Location Tracking Parameters
Location tracking examines the deviance of each frame in a video from the reference frame on a pixel by pixel basis.  For each frame it calculates the center of mass for these differences (COM) to define the center of the animal.  

`loc_thresh` : This parameter represents a percentile threshold and can take on values between 0-100.  Each frame is compared to the reference frame.  Then, to remove the influence of small fluctuations, any differences below a given percentile (relative to the maximum difference) are set to 0.  We use a value of 99.5 with good success.

`use_window` : This parameter is incredibly helpful if objects other than the animal temporarily enter the field of view during tracking (such as an experimenter’s hand manually delivering a stimulus or reward).  When use_window is set to True, a square window with the animal's position on the prior frame at its center is given more weight when searching for the animal’s location (because an animal presumably can't move far from one frame to the next).  In this way, the influence of objects entering the field of view can be avoided.  If use_window is set to True, the user should consider window_size and window_weight.

`window_size` : This parameter only impacts tracking when use_window is set to True.  This defines the size of the square window surrounding the animal that will be more heavily weighted in pixel units.  We typically set this to 2-3 times the animal’s size (if an animal is 100 pixels at its longest, we will set window_size to 200).  Note that to determine the size of the animal in pixels, the user can reference any image of the arena presented in ezTrack, which has the pixel coordinate scale on its axes.

`window_weight` : This parameter only impacts tracking when use_window is set to True.  When window_weight is set to 1, pixels outside of the window are not considered at all; at 0, they are given equal weight. Notably, setting a high value that is still not equal to 1 (e.g., 0.9) should allow ezTrack to more rapidly find the animal if, by chance, it moves out of the window.  

`method` : This parameter determines the luminosity of the object ezTrack will search for relative to the background and accepts values of 'abs', 'light', and 'dark'. Option 'abs' does not take into consideration whether the animal is lighter or darker than the background and will therefore track the animal across a wide range of backgrounds. 'light' assumes the animal is lighter than the background, and 'dark' assumes the animal is darker than the background. Option 'abs' generally works well, but there are situations in which you may wish to use the others.  For example, if a tether is being used that is opposite in color to the animal (a white wire and a black mouse), the ‘abs’ method is much more likely to be biased by the wire, whereas option ‘dark’ will look for the darker mouse.

`rmv_wire` : When rmv_wire is set to True, an algorithm is used to attempt to remove wires from the field of view.  If rmv_wire is set to True, the user should consider wire_krn.

`wire_krn` : This parameter only impacts tracking when rmv_wire is set to True. This value should be set between the width of the wire and the width of the animal, in pixel units. 

In [ ]:
tracking_params = {
    'loc_thresh'    : 99.5, 
    'use_window'    : True, 
    'window_size'   : 100, 
    'window_weight' : .9, 
    'method'        : 'abs',
    'rmv_wire'      : False, 
    'wire_krn'      : 5
}

### 8b. (Optional) Display Examples of Location Tracking to Confirm Threshold
In order to confirm threshold is working, a subset of images is analyzed and displayed using the selected `tracking_params`.  The original image is displayed on the left and the difference values to the right.  The center of mass (COM) is pinpointed on images.  Notably, because individual frames are used, window settings are not applicable here.  Because of this, actual tracking in video is likely to be better.

The user can change the number examples below as they see fit.

In [ ]:
%%output size = 100

img_exmpls = lt.LocationThresh_View(video_dict, tracking_params, examples=4)
img_exmpls.cols(2)

### 8c. Track Location and Save Results to .csv File
For each frame the location of the animal's center of mass is recorded in x/y coordinates.  If ROIs are supplied, for each frame it is determined whether the animal is in each of the ROIs.  Frame-by-frame distance is also calculated in pixel units.  This data is returned in a Pandas dataframe with columns: frame, x, y, dist, and whether the animal is in each ROI specified (True/False).  Data is saved to a .csv in the same folder as the video.  First 5 rows of data are presented.

In [ ]:
location = lt.TrackLocation(video_dict, tracking_params)   
location.to_csv(output_path[0] + "/" + prefix +'_locationOutput.csv', index=False)
location.head()

### 8d. (Optional) Display Animal Distance/Location Across Session
Below, the animals distance and location across the video is plotted.  Smooth traces are expected in the case where the animal is tracked consistently across the session.  Under heatmap, sigma controls 'binning' of location. When 'sigma = None' default value is provided; but sigma can also be set to any value.

In [ ]:
%%output size = 200

w, h = 1000,300 

plt_dist = hv.Curve((location['Frame'],location['Distance_px']),'Frame','Pixel Distance').opts(
    height=h,width=w,color='red',title="Distance Across Session",toolbar="below")
plt_trks = lt.showtrace(video_dict, location, color="red", alpha=.05, size=5)
plt_hmap = lt.Heatmap(video_dict, location, sigma=5)

(plt_trks + plt_hmap + plt_dist).cols(1)


In [ ]:
lst = []
for poly in range(len(video_dict['roi_stream'].data['xs'])):
            x = np.array(video_dict['roi_stream'].data['xs'][poly]) #x coordinates
            y = np.array(video_dict['roi_stream'].data['ys'][poly]) #y coordinates
            lst.append( [ (x[vert],y[vert]) for vert in range(len(x)) ] )

patches = []
num_polygons = len(lst)


for i in range(num_polygons):
    polygon = Polygon(lst[i], True, facecolor = "lightblue", alpha = 0.2)
    polyEdge = Polygon(lst[i], fill=False, alpha=1 ,edgecolor= "black",linewidth=1, linestyle = '--')
    patches.append(polygon)
    patches.append(polyEdge)

fig, ax = plt.subplots()
imgplot = plt.imshow(video_dict['reference'],cmap="gray")
scalebar = ScaleBar(video_dict['scale']["factor"], units="cm", color="white",box_alpha=0, location="lower right",font_properties={'size' : 20})
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
p = PatchCollection(patches, match_original=True)
plt.scatter(location['X'],location['Y'], color = "Yellow", alpha =0.05)
plt.title("Motion trace", fontsize=40)
ax.add_collection(p)
ax.autoscale()
plt.gcf().set_size_inches(15, 8)
plt.gca().add_artist(scalebar)
plt.savefig(output_path[1] + "/" + prefix +'_Motion_trace.pdf', bbox_inches='tight' )
plt.show()

In [ ]:
patches = []
num_polygons = len(lst)


for i in range(num_polygons):
    polygon = Polygon(lst[i], True, facecolor = "white", alpha = 0)
    polyEdge = Polygon(lst[i], fill=False, alpha=1 ,edgecolor= "white",linewidth=2, linestyle = '--')
    patches.append(polygon)
    patches.append(polyEdge)


heatmap = np.zeros(video_dict['reference'].shape)
for frame in range(len(location)):
    Y,X = int(location.Y[frame]), int(location.X[frame])
    heatmap[Y,X]+=1
    
sigma = 5 # change the number for heatmap resolusion 
heatmap = cv2.GaussianBlur(heatmap,(0,0),sigma)
heatmap = (heatmap / heatmap.max())*255

scalebar = ScaleBar(video_dict['scale']["factor"], units="cm", color="white",box_alpha=0, location="lower right",font_properties={'size' : 15})

fig, ax = plt.subplots()
p = PatchCollection(patches, match_original=True)
plt.imshow(heatmap, cmap="gnuplot2") # change color, google "matplotlib cmap" for details
plt.colorbar()
plt.title("Heatmap", fontsize = 40)
plt.gca().add_artist(scalebar)
ax.add_collection(p)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.gcf().set_size_inches(15, 8)
plt.savefig(output_path[2] + "/" + prefix +'_Heatmap.pdf', bbox_inches='tight')
plt.show()

In [ ]:
lst = []
for poly in range(len(video_dict['roi_stream'].data['xs'])):
    x = np.array(video_dict['roi_stream'].data['xs'][poly]) #x coordinates
    y = np.array(video_dict['roi_stream'].data['ys'][poly]) #y coordinates
    lst.append( [ (x[vert],y[vert]) for vert in range(len(x)) ] )
poly = hv.Polygons(lst).opts(fill_alpha=0.1,line_dash='dashed')

p1 = np.average(lst[2], axis=0) # center of sencond ROI, if petri dish is the thrid ROI you circled, please make the lst[2]
all_coord = np.array([location['X'],location['Y']]).T
p2 = np.average(all_coord, axis=0) # averge position
pall = np.array((p1, p2))
dist = str(round(math.dist(p1, p2)*video_dict['scale']["factor"], 2)) # distance
patches = []
num_polygons = len(lst)


for i in range(num_polygons):
    polygon = Polygon(lst[i], True, facecolor = "lightblue", alpha = 0.2)
    polyEdge = Polygon(lst[i], fill=False, alpha=1 ,edgecolor= "black",linewidth=1, linestyle = '--')
    patches.append(polygon)
    patches.append(polyEdge)

fig, ax = plt.subplots()
imgplot = plt.imshow(video_dict['reference'],cmap="gray")
scalebar = ScaleBar(video_dict['scale']["factor"], units="cm", color="white",box_alpha=0, location="lower right",font_properties={'size' : 20})
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
p = PatchCollection(patches, match_original=True)
plt.scatter(location['X'],location['Y'], color = "Yellow", alpha =0.05)
plt.scatter(p1[0],p1[1], color = "Blue", alpha =1, s=150)
plt.scatter(p2[0],p2[1], color = "Red", alpha =1, s=150)
#plt.scatter([p1[0] for p1 in pall],[p1[1] for p1 in pall],color = "Red", alpha =1, s =150)
plt.title("Distance between average location \n to odorant = "+dist + "cm", fontsize = 30)
ax.add_collection(p)
ax.autoscale()
plt.gcf().set_size_inches(15, 8)
plt.gca().add_artist(scalebar)
plt.savefig(output_path[3] + "/" + prefix +'_Motion_trace_distance.pdf', bbox_inches='tight')
plt.show()

In [ ]:
pall

In [ ]:

plt.plot(location['Frame'],location['Distance_px']*video_dict['scale']["factor"],color="red")
plt.title("Distance Across Session", fontsize = 40)
plt.xlabel("frame", fontsize = 30)
plt.ylabel("cm", fontsize = 30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.gcf().set_size_inches(15, 8)
plt.savefig(output_path[4] + "/" + prefix +'_distance_across_session.pdf', bbox_inches='tight')
plt.show()


---
# 9. (Optional) Create Binned Summary Report and Save
The code below allows the user to either save a csv containing summary data for user-defined bins (e.g. proportion of time in each region and distance travelled for each minute) or a session-wide average. 

***If you only want a session avg***, set `bin_dict = None`

***To specify bins***, set bin_dict using the following notation, where start and stop represent time in frames:

```
bin_dict = {
    'BinName1': (start, stop),
    'BinName2': (start, stop),
    'BinName3': (start, stop),
}

```

In [ ]:
bin_dict = None



summary = lt.Summarize_Location(location, video_dict, bin_dict=bin_dict)
summary['Average_position_to_dish (cm)'] = dist
summary.to_csv(output_path[5] + "/" + prefix + '_SummaryStats.csv', index=False)
summary.head()

In [ ]:

summary.head()

---
# 10. (Optional) View Video of Tracking
**Note** that tracking must be done before this (Step 7c). 

`start` : The frame video playback is to be started on. Note that this is relative to the start of tracking, where 0 is the first tracked frame.

`stop` : The frame video playback is to end on.  Note that this is relative to the start of tracking, where 0 is the first tracked frame.

`fps` : The speed of video playback.  Must be an integer.  Video playback may also be slower depending upon computer speed. 

`resize` : If the user wants the output to be larger or smaller, or they want the aspect ratio to be different, resize can be supplied as in the following example:
	`‘resize’ : (100,200)`
Here, the first number corresponds to the adjusted width of the frame, whereas the second number corresponds to the adjusted height.  Both numbers reflect pixel units and should be integers. Set resize equal to None if no resizing is to be done.

`save_video` : To save the video clip, set to True.


In [ ]:
display_dict = {
    'start'      : 0, 
    'stop'       : 1000, 
    'fps'        : 100,
    'resize'     : None,
    'save_video' : False
}

lt.PlayVideo(video_dict,display_dict,location)